In [1]:
import boto3
import os
from dotenv import load_dotenv

In [2]:
# Load the environment variables from the .env file
load_dotenv()

False

In [31]:
# Creating the DynamoDB Client
# make sure to set the AWS_ACCESS_KEY_ID and
# AWS_SECRET_ACCESS_KEY environment variables
# OR pass credentials when creating the client:
dynamodb_client = boto3.client('dynamodb', region_name="us-east-1")

TABLE_NAME = "fumehoods"

hood = {
    "id": {"S": "Biotech.Floor_4.Lab_433.Hood_1"},
    "sash_sensor": {
        "S": "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash"
    },
    "occ_sensor": {
        "S": "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ"
    },
    "temp_sensor": {
        "S": "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log"
    },
    "flow_sensor": {
        "S": "#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log"
    },
}

# Put the above data into the table
response = dynamodb_client.put_item(TableName=TABLE_NAME, Item=hood)
print("put_item response:")
display(response)

# Print what was just added
response = dynamodb_client.get_item(
    TableName=TABLE_NAME, Key={"id": {"S": "Biotech.Floor_4.Lab_433.Hood_1"}}
)
print("get_item response:")
display(response["Item"])

put_item response:


{'ResponseMetadata': {'RequestId': 'LFAADLCIPV2FHOVU6PEV36R00NVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Thu, 28 Sep 2023 18:13:11 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'LFAADLCIPV2FHOVU6PEV36R00NVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

get_item response:


{'occ_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/occ'},
 'id': {'S': 'Biotech.Floor_4.Lab_433.Hood_1'},
 'flow_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hoodvalve_flow/trend_log'},
 'temp_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/zone/zone_temp/trend_log'},
 'sash_sensor': {'S': '#biotech/biotech_4th_floor/fourth_floor_fume_hood_lab_spaces/lab_433_control/hood_sash'}}

In [32]:
dynamodb = boto3.resource('dynamodb', region_name="us-east-1")

table = dynamodb.Table('fumehoods')

print(table.key_schema)

[{'AttributeName': 'id', 'KeyType': 'HASH'}]
